In [1]:
# !pip install mysql-connector-python
# !pip show sqlalchemy

Name: SQLAlchemy
Version: 2.0.31
Summary: Database Abstraction Library
Home-page: https://www.sqlalchemy.org
Author: Mike Bayer
Author-email: mike_mp@zzzcomputing.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: greenlet, typing-extensions
Required-by: bigframes, ipython-sql


In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, inspect

[데이터타입 통합]
kopo_channel_seasonality_new.csv 파일을

불러온 후selloutData 변수에 담으세요 이후
QTY컬럼→실수(float),
이외컬럼→문자(str)로 변경하세요

In [3]:
selloutData = pd.read_csv("https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/kopo_channel_seasonality_new.csv")

In [4]:
targetRegion=["A01","A02","A03"]

In [5]:
selloutData = selloutData.loc[ selloutData.REGIONID.isin(targetRegion) ]

In [6]:

# [데이터 타입통합 실습문제]
# QTY: float 이외에는 str으로 변환하세요

columnList = list(selloutData.columns)
for i in range(0,len(columnList)):
  selloutData[columnList[i]] = selloutData[columnList[i]].astype(str)

selloutData.QTY = selloutData.QTY.astype(float)

# [데이터 조작 실습문제]
# [불량 데이터 처리]
# kopo_channel_seasonality_new.csv 자료를 담은
# selloutData 변수에서
# QTY컬럼 음수(반품)인 경우 0, 양수인 경우 기존 QTY 값
# 유지하는 로직을 적용하여 QTY_NEW 컬럼을 추가하세요

# QTY 음수 제거
selloutData["QTY_NEW"] = np.where( selloutData.QTY < 0, 0, selloutData.QTY)
selloutData.loc[ (selloutData.QTY < 0) &
                 (selloutData.QTY_NEW != 0)]
selloutData.loc[ (selloutData.QTY >= 0) &
                 (selloutData.QTY_NEW != selloutData.QTY)]

# 연주차 컬럼 생성 및 특정 조회조건에 따른 데이터 제거
selloutData["YEAR"] = selloutData.YEARWEEK.str[:4]
selloutData["WEEK"] = selloutData.YEARWEEK.str[4:]
refinedSelloutData = selloutData.loc[ selloutData.WEEK.astype(int) <= 52 ]

# [정렬 실습문제]
# refinedSelloutData 에서 →
# 지역, 상품, 연주차 컬럼순으로 오름차순 정렬하여
# sortedData 변수에 담으세요

sortKey = ["REGIONID", "PRODUCT", "YEARWEEK"]
sortedData = refinedSelloutData.sort_values(by=sortKey, ascending = True)
sortedData.head(2)

# [그룹바이 실습문제]]
# [지역, 상품, 연도 별 집계]
# sortedData 에서 지역, 상품, 연도 단위
# 판매량(QTY_NEW) 의 평균 연산 후
# groupData 변수에 담으세요
# 이후 컬럼명을 QTY_MEAN로 변경하세요

groupKey = ["REGIONID", "PRODUCT", "YEAR"]
groupData = sortedData.groupby(by=groupKey)["QTY_NEW"].agg(["mean"]).reset_index()

groupData = groupData.rename(columns= {"mean":"QTY_MEAN"} )
groupData.head(5)

,REGIONID,PRODUCT,YEAR,QTY_MEAN
0,A01,PRODUCT1,2014,8622.461538
1,A01,PRODUCT1,2015,6384.788462
2,A01,PRODUCT1,2016,2131.076923
3,A01,PRODUCT12,2014,169397.826923
4,A01,PRODUCT12,2015,195479.884615


refinedSelloutData와 groupData를
[REGIONID, PRODUCT, YEAR] 키로 조인하여
mergedData 변수에 아래와 같이 담으세요

In [7]:
joinKey = ["REGIONID","PRODUCT","YEAR"]

In [8]:
mergedData = pd.merge( left=refinedSelloutData,
          right = groupData,
          on = joinKey,
          how = "left" )

In [9]:
# mergedData.sort_values(by=sortKey)

In [10]:
mergedData.shape

(8892, 8)

mergedData를 활용하여
SEASONALITY = QTY_NEW / QTY_MEAN
SEASONALITY 컬럼을 생성한다.

In [11]:
# np.where 활용 풀이
mergedData["SEASONALITY"] = np.where( mergedData.QTY_MEAN == 0 , 0, mergedData.QTY_NEW / mergedData.QTY_MEAN)

In [12]:
# function 활용 풀이
def calculate_seasonality(row):
    if row['QTY_MEAN'] != 0:
        return row['QTY_NEW'] / row['QTY_MEAN']
    else:
        return 0

# SEASONALITY 컬럼 생성
mergedData['SEASONALITY'] = mergedData.apply(calculate_seasonality, axis=1)

마지막으로 [지역, 상품, 주차] 별
SEASONALITY 평균 값을
산출하여 finalResult 변수에 담으세요


In [13]:
groupkeys=['REGIONID', 'PRODUCT', 'WEEK']
finalResult = mergedData.groupby(by=groupkeys)["SEASONALITY"].agg( "mean" ).reset_index()
finalResult

,REGIONID,PRODUCT,WEEK,SEASONALITY
0,A01,PRODUCT1,01,2.190485
1,A01,PRODUCT1,02,1.643770
2,A01,PRODUCT1,03,2.240885
3,A01,PRODUCT1,04,1.914736
4,A01,PRODUCT1,05,1.754313
...,...,...,...,...
2959,A03,PRODUCT63,48,1.026664
2960,A03,PRODUCT63,49,1.475198
2961,A03,PRODUCT63,50,2.085499
2962,A03,PRODUCT63,51,1.725693


In [15]:
finalResult.to_csv("./season_result.csv")

In [14]:
# # MySQL 데이터베이스 연결 정보 설정 (클라우드)
# host = "54.180.133.247"
# port = "3306"
# user = "kopouser"
# password = "kopouser"
# database = "kopo"

# # SQLAlchemy 엔진 생성
# engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}')

# # 데이터프레임을 MySQL 데이터베이스의 테이블에 저장
# table_name = 'seasonality_result_0802'
# finalResult.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

KeyboardInterrupt: 

In [ ]:
# groupKey = ["REGIONID","PRODUCT"]
# def groupMa5(eachGroup):
#     """
#     정의: 데이터를 받아서 이동평균 산출
#     """
#     # eachGroup = groupMaData.get_group( list( groupMaData.groups )[0] )
#     eachGroup["MA5"] = eachGroup.QTY_NEW.rolling(window=5, center=True, min_periods=1).mean()
#     return eachGroup
# groupMaResult = sortedData.groupby(groupKey).apply(groupMa5)
# groupMaResult = groupMaResult.reset_index(drop=True)
# joinKey = ["REGIONID","PRODUCT","YEARWEEK"]
# mergedData = pd.merge( left = refinedSelloutData,
#           right = groupMaResult,
#           on=joinKey,
#           how="left")

In [ ]:
# # IDX 컬럼 생성 후 조회하기
# groupMaResult["IDX"] = groupMaResult.REGIONID+"_"+groupMaResult.PRODUCT+"_"+groupMaResult
# groupMaResult
# groupMaResult.loc[ groupMaResult.NEW_COL == "A60_PRODUCT4"]